# Загрузка базы вопросов ЧГК

Нужно загрузить все доступные вопросы и ответы к ним с базы http:///db.chgk.info . Результаты необходимо сохранить в каком-либо формате в форме вопрос-ответ (pickle, sqlite, текстовый файл и т.д.). 

In [ ]:
import os
import codecs
import urllib3
from bs4 import BeautifulSoup

base_url_beginning = 'https://db.chgk.info'
base_url_ending = '/last?page='
page_count = 40
page_num = 1

http = urllib3.PoolManager()

dir_path = os.path.dirname(os.path.realpath(__file__))

text = open('text.txt', 'w', encoding = 'utf-8')

for page in range(1, page_count + 1):
    page_url = base_url_beginning + base_url_ending + str(page_num)
    response = http.request('GET', page_url)
    toursSoup = BeautifulSoup(response.data, 'html.parser')
    linksSoup = toursSoup.find('table', {'class': 'last_packages'}).findAll('a', {'rel': 'nofollow'})
    for linkSoup in linksSoup:
        link = linkSoup['href']
        full_url = base_url_beginning + link
        response = http.request('GET', full_url)
        soup = BeautifulSoup(response.data, 'html.parser')
        elementsSoup = soup.findAll('div', {'class': 'question'})
        questions_list = []
        questionsSoup = []
        for elementSoup in elementsSoup:
            questionsSoup.append(elementSoup.find('p'))
        for questionSoup in questionsSoup:
            question = questionSoup.text
            questions_list.append(question)
        answers_list = []
        answersSoup = []
        for elementSoup in elementsSoup:
            answersSoup.append(elementSoup.find('div', {'class': 'collapsible collapsed'}).find('p'))
        for answerSoup in answersSoup:
            answer = answerSoup.text
            answers_list.append(answer)

        for i in range(0, len(questions_list)):
            text.write(questions_list[i] + '\n' + answers_list[i] + '\n''\n')

text.close()
